In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import numpy as np

### Load and handle missing data

In [2]:
df = pd.read_csv('weather_by_time.csv')

In [4]:
pivoted = df.copy(deep=True)

In [5]:
pivoted['month'] = pd.to_datetime(pivoted['day']).dt.month
pivoted['year'] = pd.to_datetime(pivoted['day']).dt.year
pivoted['day'] = pd.to_datetime(pivoted['day']).dt.dayofyear

In [6]:
pivoted = pivoted.set_index(['year', 'month', 'day', 'hour'])

In [7]:
print("Originally contains data from approx " + str(len(df.columns) / 5) + " weather stations")

Originally contains data from approx 566.8 weather stations


***Drop stations with more than or equal to 45 days of missing data***

In [8]:
by_day_df = pivoted.groupby(['year', 'month', 'day']).mean()
by_day_df = by_day_df.dropna(thresh=len(by_day_df) - 44, axis=1)
pivoted = pivoted[pivoted.columns.intersection(by_day_df.columns)]

In [9]:
print("Used data from approx " + str(len(pivoted.columns) / 5) + " weather stations")

Used data from approx 435.2 weather stations


In [10]:
by_day_df.describe()

,air-temp|72011354829,air-temp|72013704867,air-temp|72017063851,air-temp|72019854813,air-temp|72026654809,air-temp|72027504872,air-temp|72027763843,air-temp|72027803704,air-temp|72028404877,air-temp|72028503734,...,wind_speed|74671013806,wind_speed|74671693808,wind_speed|74692503741,wind_speed|74693093737,wind_speed|74693503709,wind_speed|74693603712,wind_speed|74694003708,wind_speed|99999903755,wind_speed|99999993816,wind_speed|A0001704868
count,1094.000000,1094.000000,1079.000000,1088.000000,1095.000000,1094.000000,1094.000000,1092.000000,1095.000000,1084.000000,...,1094.000000,1095.000000,1069.000000,1092.000000,1061.000000,1072.000000,1053.000000,1056.000000,1084.000000,1083.000000
mean,11.253638,10.991222,15.061150,6.434981,10.515484,11.162786,15.865105,14.771681,9.884497,15.286746,...,3.544125,3.405704,3.175638,3.085240,2.759217,2.712839,3.127027,3.881214,3.443570,3.470446
std,10.708976,10.932421,9.738313,10.235087,10.354762,10.505338,8.325511,8.931806,10.375531,9.148075,...,1.194307,1.118219,1.109362,0.993812,0.847960,0.800128,1.079341,1.425698,1.275627,1.181023
min,-16.552174,-23.031818,-13.270833,-19.617391,-21.043478,-16.941667,-7.266667,-13.695652,-19.026087,-12.954167,...,1.228571,1.650000,1.500000,0.576923,1.500000,1.500000,1.500000,1.633333,1.500000,1.500000
25%,2.116667,2.224321,7.373913,-1.622826,1.565217,2.518478,9.140761,7.258152,1.082880,7.521875,...,2.667647,2.602381,2.464286,2.417984,2.141667,2.155303,2.350000,2.742143,2.529003,2.609611
50%,10.903351,11.645109,16.182609,5.878261,11.217391,11.454982,17.241304,16.000000,9.970833,16.156250,...,3.320238,3.170000,2.957895,2.888478,2.577778,2.545804,2.929167,3.619112,3.134167,3.200000
75%,21.131703,20.832337,24.039130,15.765217,19.717391,20.822826,23.223295,22.652174,19.397205,23.340625,...,4.119545,3.978938,3.671429,3.524436,3.173333,3.123974,3.695000,4.785489,4.075694,4.093152
max,30.373913,28.869565,31.766667,26.182609,29.173913,31.285714,29.918182,29.375000,28.612500,30.637500,...,8.860870,9.269565,17.000000,10.408696,8.070833,8.285714,11.240000,8.654167,9.045455,8.078261


***Fill in missing data*** 

Fill in full days of missing data by taking the average of the day before and after where there were data 

In [9]:
# Create a df where True = there is no data for the entire day
    #False = there is data for at least 1 hour of the day 
test = by_day_df.copy(deep=True)
test[test.notnull()] = False
test[test.isnull()] = True

In [10]:
# Create a df with just year, month, day, and hour 
time_col = pd.DataFrame(index=pivoted.index)
time_col = time_col.reset_index()

In [11]:
# Add hour to the index of the Boolean df (True = null value) 
# Essentially copying the value for a day over 24 hours 
test = pd.merge(test, time_col, on=['year', 'month', 'day'])
test = test.set_index(['year', 'month', 'day', 'hour'])

In [14]:
fill_missing_days = (by_day_df.ffill()+by_day_df.bfill())/2
fill_missing_days = fill_missing_days.bfill().ffill()

In [ ]:
# kernel always dies here

# Take the daily data and copy it 
fill_missing_days = fill_missing_days.reset_index()
avg_day_with_hourly_data = pd.merge(fill_missing_days, time_col, on=['year', 'month', 'day'])
avg_day_with_hourly_data = avg_day_with_hourly_data.set_index(['year', 'month', 'day', 'hour'])

In [ ]:
avg_day_with_hourly_data[test==False] = False

In [ ]:
pivoted = pivoted.fillna(avg_day_with_hourly_data)
pivoted = pivoted.apply(pd.to_numeric)

In [1]:
by_hour = (pivoted.ffill()+pivoted.bfill())/2
by_hour = by_hour.bfill().ffill()

NameError: name 'pivoted' is not defined

In [ ]:
weather_df = (pivoted.ffill()+pivoted.bfill())/2
weather_df = weather_df.bfill().ffill()
weather_df

### Renewable Data

In [209]:
target = pd.read_csv('/root/renewables.csv', parse_dates=['datetime_beginning_utc'])

In [214]:
target['hour'] = target['datetime_beginning_utc'].dt.hour
target['month'] = pd.to_datetime(target['datetime_beginning_utc']).dt.month
target['year'] = pd.to_datetime(target['datetime_beginning_utc']).dt.year
target['day'] = pd.to_datetime(target['datetime_beginning_utc']).dt.dayofyear

target = target.drop(['Unnamed: 0', 'total_mw', 'datetime_beginning_utc'], axis=1)

In [160]:
target = target.pivot_table(values=['mw'], index=['year', 'month', 'day', 'hour'], columns='fuel_type')
target.columns = target.columns.map('|'.join).str.strip('|')

In [162]:
# Drop hydro and other renewables for now
target = target.drop(['mw|Hydro', 'mw|Other Renewables'], axis=1)

solar = target[['mw|Solar']]
wind = target[['mw|Wind']]
wind

mw|Wind
year month day hour         
2016 1     1   5      2263.5
               6      2366.4
               7      2543.3
               8      2244.9
               9      2287.8
...                      ...
2018 12    365 1      4209.1
               2      4161.8
               3      4327.9
               4      4684.2
               5      4179.7

[26108 rows x 1 columns]

### Split into a separate dataframe for each metric, standardize, apply PCA

Metrics: 
- Air temp 
- Dew point temp: measure of how much water vapor is in the air 
- Sea level pressure: atmospheric pressure at sea level at a given location.
- Sky ceiling height: height of the lowest layer of clouds above the surface that are either broken or overcast, but not thin
- Wind speed 

### Pivot target (solar, wind) so it's by hour

In [165]:
solar_hour = solar.reset_index()
solar_hour = solar_hour.pivot_table(index = ['year','month', 'day'], columns="hour", values=solar_hour.columns)
solar_hour.columns = solar_hour.columns.map('{0[0]}|{0[1]}'.format)
solar_hour = solar_hour.dropna()

In [262]:
wind_hour = wind.reset_index()
wind_hour = wind_hour.pivot_table(index = ['year','month', 'day'], columns="hour", values=wind_hour.columns)
wind_hour.columns = wind_hour.columns.map('{0[0]}|{0[1]}'.format)
wind_hour = wind_hour.dropna()

### Pivot features so it's by hour

In [170]:
weather_df_hour = weather_df.reset_index()
weather_df_hour = weather_df_hour.pivot_table(index = ['year','month', 'day'], columns="hour", values=weather_df_hour.columns)
weather_df_hour.columns = weather_df_hour.columns.map('{0[0]}|{0[1]}'.format)
weather_df_hour = weather_df_hour.dropna()
weather_df_hour

air-temp|72011354829|0  air-temp|72011354829|1  \
year month day                                                   
2016 1     2                      -2.5                    -2.2   
           3                      -0.4                    -0.2   
           4                      -1.5                    -2.2   
           5                      -7.6                    -8.1   
           6                      -3.2                    -3.4   
...                                ...                     ...   
2018 12    361                     3.2                     2.5   
           362                     5.0                     6.0   
           363                     2.5                     1.9   
           364                    -1.5                    -1.3   
           365                     0.0                     0.0   

                air-temp|72011354829|2  air-temp|72011354829|3  \
year month day                                                   
2016 1     2                      -2.0                    -2.0   
           3                      -0.5                    -0.4   
           4                      -2.9                    -3.5   
           5                      -8.4                    -8.6   
           6                      -3.6                    -3.8   
...                                ...                     ...   
2018 12    361                     2.0                     1.6   
           362                     6.8                     7.1   
           363                     2.0                     2.0   
           364                    -1.1                    -1.5   
           365                    -0.4                    -0.3   

                air-temp|72011354829|4  air-temp|72011354829|5  \
year month day                                                   
2016 1     2                       0.0                    -2.0   
           3                       0.0                     0.7   
           4                      -3.5                    -3.4   
           5                       0.0                    -9.0   
           6                      -4.1                    -4.1   
...                                ...                     ...   
2018 12    361                     0.0                     1.1   
           362                     0.0                     8.0   
           363                     0.0                     1.7   
           364                     0.0                    -1.5   
           365                     0.0                    -0.8   

                air-temp|72011354829|6  air-temp|72011354829|7  \
year month day                                                   
2016 1     2                      -1.8                    -1.8   
           3                       1.0                     1.1   
           4                      -3.2                    -3.3   
           5                      -9.2                    -9.4   
           6                      -4.7                    -4.8   
...                                ...                     ...   
2018 12    361                     1.4                     1.3   
           362                     8.5                     9.0   
           363                     1.5                     1.2   
           364                    -1.0                    -0.8   
           365                    -1.0                    -0.7   

                air-temp|72011354829|8  air-temp|72011354829|9  ...  \
year month day                                                  ...   
2016 1     2                      -2.1                    -2.1  ...   
           3                       1.1                     0.8  ...   
           4                      -3.5                    -3.9  ...   
           5                     -10.0                   -10.2  ...   
           6                      -5.1                    -6.0  ...   
...                                ...                     ...  ...   
2018 12    3

In [263]:
# Specify if you're doing wind or solar
merged = weather_df_hour.merge(wind_hour, left_index=True, right_index=True, how='inner')

In [264]:
X = merged.iloc[:,:-24]
y = merged[merged.columns[-24:]]

In [265]:
train_pct_index = int(0.8 * len(y))
Y_train, Y_test = y[:train_pct_index], y[train_pct_index:]

In [240]:
# Standardize 
def split(df):
    # 80-20 split 
    df = df.copy()
    
    train_pct_index = int(0.8 * len(df))
    X_train, X_test = df[:train_pct_index], df[train_pct_index:]

    # Standardize features 
#     scaler = StandardScaler()
#     scaler.fit(X_train.values) # fit on training set 

#     scaled_X_train = scaler.transform(X_train.values)
#     scaled_X_test = scaler.transform(X_test.values)

# Convert back to df after scaling 
#     X_train = pd.DataFrame(scaled_X_train, index=X_train.index, columns=X_train.columns)
#     X_test = pd.DataFrame(scaled_X_test, index=X_test.index, columns=X_test.columns)
    return X_train, X_test

X_train, X_test = split(X)

In [243]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import copy 

def split_apply_pca(X_train, X_test, num_components):
    # PCA Analysis
    pca = PCA(n_components = num_components)
    pca.fit(X_train.values)

    X_train_pca = pca.transform(X_train.values)
    X_test_pca = pca.transform(X_test.values)
    
    # Convert back to df after pca 
    X_train = pd.DataFrame(X_train_pca, index=X_train.index)
    X_test = pd.DataFrame(X_test_pca, index=X_test.index)
    
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_.cumsum()))

    return X_train, X_test 

In [244]:
def pca_by_metric(X_train, X_test, metric_string, num_components):
    X_train = X_train.loc[:, X_train.columns.str.startswith(metric_string)]
    X_test = X_test.loc[:, X_test.columns.str.startswith(metric_string)]
    X_train, X_test = split_apply_pca(X_train, X_test, num_components)
    X_train.columns = [metric_string + '|' + str(col) for col in X_train.columns]
    X_test.columns = [metric_string + '|' + str(col) for col in X_test.columns]
    return X_train, X_test

In [246]:
air_temp_df_train, air_temp_df_test = pca_by_metric(X_train, X_test , 'air-temp', 3)

Explained variation per principal component: [0.84657874 0.8890236  0.90315349]


In [247]:
dew_point_temp_df_train, dew_point_temp_df_test = pca_by_metric(X_train, X_test , 'dew_point_temp', 3)

Explained variation per principal component: [0.85220282 0.8993575  0.9160058 ]


In [255]:
sea_level_pressure_df_train, sea_level_pressure_df_test = pca_by_metric(X_train, X_test , 'sea_level_pressure', 25)

Explained variation per principal component: [0.02366586 0.04007393 0.05530849 0.06749286 0.07755279 0.08620622
 0.09400469 0.10162169 0.10840657 0.11515109 0.12117887 0.12689749
 0.13253059 0.13782165 0.14275046 0.14751503 0.15207686 0.15655194
 0.16088082 0.1651044  0.16919113 0.1732399  0.17720606 0.18099905
 0.18477023]


In [253]:
sky_ceiling_height_df_train, sky_ceiling_height_df_test = pca_by_metric(X_train, X_test , 'sky_ceiling_height', 25)

Explained variation per principal component: [0.21593852 0.30300742 0.35738269 0.39140275 0.41396811 0.43199294
 0.44675281 0.45906681 0.47013163 0.4798338  0.48892733 0.49722902
 0.50391114 0.51017233 0.51603323 0.52182628 0.52714478 0.53229519
 0.53672586 0.5407499  0.54450335 0.54823406 0.55192204 0.55542974
 0.55881159]


In [254]:
wind_speed_df_train, wind_speed_df_test = pca_by_metric(X_train, X_test , 'wind_speed', 25)

Explained variation per principal component: [0.23876318 0.33854408 0.38156636 0.41756863 0.44494356 0.4684719
 0.48190321 0.4945401  0.50536882 0.51603449 0.52557539 0.5338145
 0.54073527 0.54727363 0.55314071 0.55882178 0.5641681  0.56892123
 0.57363189 0.57799606 0.58209191 0.58610366 0.58988656 0.59344185
 0.59684556]


In [256]:
X_train = pd.concat([air_temp_df_train, dew_point_temp_df_train, sea_level_pressure_df_train, sky_ceiling_height_df_train, wind_speed_df_train], axis=1, join='inner')
X_test = pd.concat([air_temp_df_test, dew_point_temp_df_test, sea_level_pressure_df_test, sky_ceiling_height_df_test, wind_speed_df_test], axis=1, join='inner')

In [ ]:
### DOWNLOAD HERE (without circular encoding )

### Circular encoding for time data

In [257]:
# Circular encoding of day of year and month 
def sin_cos_correction(df):
    df = df.reset_index()
    
    df['day_sin'] = np.sin((df['day']-1)*(2.*np.pi/365))
    df['day_cos'] = np.cos((df['day']-1)*(2.*np.pi/365))
    df['month_sin'] = np.sin((df['month']-1)*(2.*np.pi/12))
    df['month_cos'] = np.cos((df['month']-1)*(2.*np.pi/12))
    
    df = df.drop(['month', 'day'], axis=1)
    df = df.set_index(['year', 'month_sin', 'month_cos', 'day_sin', 'day_cos'])
    return df

In [ ]:
X_train = sin_cos_correction(X_train)
X_test = sin_cos_correction(X_test)

In [267]:
Y_train = sin_cos_correction(Y_train)
Y_test = sin_cos_correction(Y_test)

### Standardize

In [259]:
# Standardize features 
scaler = StandardScaler()
scaler.fit(X_train.values) # fit on training set 

scaled_X_train = scaler.transform(X_train.values)
scaled_X_test = scaler.transform(X_test.values)

# Convert back to df after scaling 
X_train = pd.DataFrame(scaled_X_train, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaled_X_test, index=X_test.index, columns=X_test.columns)
    

### Save to csv

In [260]:
X_train.to_csv("X_Train.csv")
X_test.to_csv("X_test.csv")

In [1]:
Y_test

NameError: name 'Y_test' is not defined

In [268]:
Y_train.to_csv("y_Train.csv")
Y_test.to_csv("y_test.csv")